In [ ]:
import os
import fnmatch
import shutil

# ==============================================================================
# CONFIGURATION - CHANGE THESE VALUES
# ==============================================================================

# 1. The full path to the folder you want to process.
#    On Windows: r"C:\Users\YourUser\Documents\MyFolder"
#    On macOS/Linux: "/home/youruser/documents/my_folder"
target_folder = r"C:\path\to\your\source_folder" # <--- SET YOUR SOURCE FOLDER

# 2. The path where the converted .txt files will be saved.
#    ✨ Using an output folder is the SAFEST option as it leaves original files untouched.
#    If this is set to None, the script will rename the original files IN-PLACE.
#    Example: r"C:\path\to\your\output_folder"
output_folder = r"C:\path\to\your\output_folder" # <--- SET YOUR OUTPUT FOLDER

# 3. Set to True to process files in all subfolders, or False to only process
#    files in the top-level folder.
include_subfolders = True

# 4. List of file extensions to exclude from conversion.
#    Uses wildcard matching (e.g., "*.mp4" matches any file ending in .mp4).
excluded_extensions = ["*.wav", "*.mp4", "*.exe", "*.jpg", "*.png"]

# 5. List of folder names to completely skip during processing.
#    If a folder's name is in this list, neither it nor any of its
#    subfolders will be processed.
excluded_folders = ["venv", ".git", "temp_files", "__pycache__"]

# ==============================================================================
# SCRIPT LOGIC - NO NEED TO EDIT BELOW THIS LINE
# ==============================================================================

def convert_files_in_folder(root_path, output_path, process_subfolders, ext_exclusions, folder_exclusions):
    """
    Copies files from a source directory to an output directory and renames them
    to have a .txt extension. Includes options for recursion and exclusions.
    If output_path is None, it renames files in-place.
    """
    if not os.path.isdir(root_path):
        print(f"❌ Error: The source folder '{root_path}' does not exist. Please check the path.")
        return

    # Determine the mode of operation
    if output_path:
        mode = 'copy'
        os.makedirs(output_path, exist_ok=True) # Ensure the output folder exists
        print(f"🚀 Starting process in 'Copy Mode'.")
        print(f"   Source: '{root_path}'")
        print(f"   Destination: '{output_path}'")
    else:
        mode = 'rename'
        print(f"🚀 Starting process in 'Rename In-Place Mode'.")
        print(f"   ⚠️ Warning: Files in '{root_path}' will be permanently renamed.")
    
    print("-" * 50)

    processed_count = 0
    skipped_count = 0
    warning_count = 0
    
    for dirpath, dirnames, filenames in os.walk(root_path):
        
        # Feature: Exclude specific folders from traversal
        dirnames[:] = [d for d in dirnames if d not in folder_exclusions]

        for filename in filenames:
            original_file_path = os.path.join(dirpath, filename)
            
            # --- Exclusion Checks ---
            if filename.endswith('.txt'):
                continue

            is_excluded = any(fnmatch.fnmatch(filename.lower(), pattern.lower()) for pattern in ext_exclusions)
            if is_excluded:
                # This check is silent to avoid cluttering the output. 
                # To see skipped files, add a print statement here.
                skipped_count += 1
                continue

            # --- Path and Naming Logic ---
            new_filename = f"{filename}.txt"
            
            try:
                if mode == 'copy':
                    # Recreate the directory structure in the output folder
                    relative_dir = os.path.relpath(dirpath, root_path)
                    output_dir = os.path.join(output_path, relative_dir)
                    os.makedirs(output_dir, exist_ok=True)
                    
                    destination_path = os.path.join(output_dir, new_filename)
                    action_verb = "Copied"
                    
                    if os.path.exists(destination_path):
                        print(f"⚠️ Warning: '{destination_path}' already exists. Skipping copy.")
                        warning_count += 1
                        continue

                    shutil.copy2(original_file_path, destination_path)

                else: # mode == 'rename'
                    destination_path = os.path.join(dirpath, new_filename)
                    action_verb = "Renamed"

                    if os.path.exists(destination_path):
                        print(f"⚠️ Warning: '{destination_path}' already exists. Skipping rename.")
                        warning_count += 1
                        continue
                        
                    os.rename(original_file_path, destination_path)

                print(f"✅ {action_verb}: '{filename}' -> '{new_filename}'")
                processed_count += 1

            except (OSError, shutil.Error) as e:
                print(f"❌ Error processing '{filename}': {e}")
                warning_count += 1

        if not process_subfolders:
            break
            
    print("-" * 50)
    print("🎉 Process Complete!")
    print(f"   - Files Processed: {processed_count}")
    print(f"   - Files Skipped (due to exclusion): {skipped_count}")
    print(f"   - Warnings/Errors: {warning_count}")

# --- Execute the Function ---
convert_files_in_folder(
    root_path=target_folder,
    output_path=output_folder,
    process_subfolders=include_subfolders,
    ext_exclusions=excluded_extensions,
    folder_exclusions=excluded_folders
)

🚀 Starting process in 'Copy Mode'.
   Source: 'F:\OneDrive - Green Energy\Backup\Website\GE\NEW SITE'
   Destination: 'F:\OneDrive - Green Energy\Backup\Website\GE\NEW SITE\texts'
--------------------------------------------------
✅ Copied: 'index.html' -> 'index.html.txt'
✅ Copied: 'script.js' -> 'script.js.txt'
✅ Copied: 'sitemap.html' -> 'sitemap.html.txt'
✅ Copied: 'style - red.css' -> 'style - red.css.txt'
✅ Copied: 'style.css' -> 'style.css.txt'
✅ Copied: 'thank-you.html' -> 'thank-you.html.txt'
✅ Copied: '_contact_popup.html' -> '_contact_popup.html.txt'
✅ Copied: '_footer.html' -> '_footer.html.txt'
✅ Copied: '_header.html' -> '_header.html.txt'
✅ Copied: '_scroll_top.html' -> '_scroll_top.html.txt'
✅ Copied: '_topbar.html' -> '_topbar.html.txt'
--------------------------------------------------
🎉 Process Complete!
   - Files Processed: 11
   - Files Skipped (due to exclusion): 0
   - Warnings/Errors: 0
